In [3]:
from pathlib import Path
from vgio.quake.bsp import Bsp
import pyvista as pv
import numpy as np
from dotdict import dotdict

quake_root = Path("D:\\Games\\quake")

pv.set_jupyter_backend('ipyvtklink')  
# pv.set_jupyter_backend('pythreejs')  

content_types = {}
content_types[-1]=dotdict({"contents":"CONTENTS_EMPTY"})
content_types[-2]=dotdict({"contents":"CONTENTS_SOLID"})
content_types[-3]=dotdict({"contents":"CONTENTS_WATER"})
content_types[-4]=dotdict({"contents":"CONTENTS_SLIME"})
content_types[-5]=dotdict({"contents":"CONTENTS_LAVA"})
content_types[-6]=dotdict({"contents":"CONTENTS_SKY"})

def classify_point(plane, p):
   n = plane.normal
   return n[0]*p[0] + n[1]*p[1] + n[2]*p[2] > plane.distance and 1 or 0

def bsp_locate(node,pos):
    while not node.contents:
        node = node.children[classify_point(node.plane,pos)]    
    return node

def read_bsp(name):
   with Bsp.open(f"D:\\Games\\quake\\id1\\maps\\{name}.bsp") as bsp_file:
      planes = bsp_file.planes
      meshes = bsp_file.meshes()            
      clipnodes = bsp_file.clip_nodes

      # main model
      model = bsp_file.models[0]

      hulls = []
      for node in clipnodes:
         hulls.append(dotdict({
            "plane": planes[node.plane_number],
            "children": [node.children[0], node.children[1]]
         }))
      for node in hulls:
         def attach_node(side):
            children = node.children
            id = children[side] 
            if id<0:
               children[side] = content_types[id]
            else:
               children[side] = hulls[id]
         attach_node(0)
         attach_node(1)
      # display 32 unit bsp
      def dump_hull(node):
         content = node.get("content", None)
         if content:
            print(content)
         else:
            children = node.get("children")
            dump_hull(children[0])
            dump_hull(children[1])
      # dump_hull(hulls[bsp_file.models[0].head_node[1]])

      tris = []
      for t in meshes[0].triangles:
         l = [len(t)]
         l.extend(t)
         tris += l
      return pv.PolyData(np.array(meshes[0].vertices), np.hstack(tris)), model, hulls

pdata, model, hulls = read_bsp("lifts")
pl = pv.Plotter()
# base geometry
pl.add_mesh(pdata, style='wireframe')

inside = []
root_node = hulls[model.head_node[0]]
bmin = model.bounding_box_min
bmax = model.bounding_box_max
total = 0
for x in range(int(bmin[0]), int(bmax[0]), 8):
   for y in range(int(bmin[1]), int(bmax[1]), 8):
      for z in range(int(bmin[2]), int(bmax[2]), 8):
         pos = [float(x),float(y),float(z)]
         total += 1
         if bsp_locate(root_node,[x,y,z]).contents!=-2:
            inside.append(pos)
print(f"Solid points: {len(inside)}/{total}")
pempty = pv.PolyData(np.array(inside))
pl.add_mesh(pempty, point_size=1.0)

pl.show_grid()
pl.show()


Solid points: 264355/264355


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)